In [13]:
from IPython.display import clear_output

In [14]:
#%pip install azure-ai-evaluation
clear_output()

# Evaluating Response Quality with RelevanceEvaluator

In [15]:
import os
from azure.ai.evaluation import RelevanceEvaluator
from dotenv import load_dotenv

load_dotenv('../.env')

# Configure the AOAI model that will be used for evaluation (AI-as-a-judge)
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

# Initialize the Relevance evaluator
relevance_evaluator = RelevanceEvaluator(model_config)

# Evaluate a single query-response pair
result = relevance_evaluator(
    query="What is the capital of France?",
    response="The capital of France is Paris.",
)

print(result)

{'relevance': 4.0, 'gpt_relevance': 4.0, 'relevance_reason': 'The response is accurate and directly answers the query, providing all necessary information.'}
{'coherence': 4.0, 'gpt_coherence': 4.0, 'coherence_reason': 'The response is coherent because it directly and correctly answers the query without any unnecessary information or confusion.'}
{'fluency': 1.0, 'gpt_fluency': 1.0, 'fluency_reason': 'The input Data should get a Score of 1 because it shows minimal command of the language, with no grammatical structure or coherent message.'}
{'groundedness': 5.0, 'gpt_groundedness': 5.0, 'groundedness_reason': 'The response accurately and completely answers the query using information from the context, without introducing any errors or unrelated details.'}
{'response_length': 13}
{'contains_blocked_word': True}
{'helpfulness': 1.0, 'helpfulness_reason': 'The RESPONSE is entirely unhelpful as it does not address the philosophical nature of the QUERY or relate to the CONTEXT. It provides 

# Evaluating Response Quality with CoherenceEvaluator

In [16]:
from azure.ai.evaluation import CoherenceEvaluator

coherence_evaluator = CoherenceEvaluator(model_config)

result = coherence_evaluator(
    query="What's the capital of France?", 
    response="Paris."
)
print(result)

# Evaluating Response Quality with FluencyEvaluator

In [17]:
from azure.ai.evaluation import FluencyEvaluator

fluency_evaluator = FluencyEvaluator(model_config)

result = fluency_evaluator(
    response="Paris."
)
print(result)

In [18]:
from azure.ai.evaluation import GroundednessEvaluator

groundedness_evaluator = GroundednessEvaluator(model_config)

result = groundedness_evaluator(
    query="Who discovered penicillin?",
    context="Alexander Fleming discovered penicillin in 1928 while working at St. Mary's Hospital in London.",
    response="Alexander Fleming discovered penicillin in 1928.",
)
print(result)

# Creating custom evaluators

## Code-based evaluator

### Function-based evaluator

In [19]:
# Custom evaluator function to calculate response length
def response_length_evaluator(response, **kwargs):
    return {"response_length": len(response)}

# Example usage
result = response_length_evaluator(response="Hello, world!")
print(result)

### Class-based evaluator

In [20]:
# Custom class-based evaluator to check for blocked words
class BlocklistEvaluator:
    def __init__(self, blocklist):
        self.blocklist = blocklist

    def __call__(self, *, response: str, **kwargs):
        contains_blocked_word = any(word in response for word in self.blocklist)
        return {"contains_blocked_word": contains_blocked_word}
    
# Example usage
blocklist_evaluator = BlocklistEvaluator(blocklist=["bad", "evil", "worst"])
result = blocklist_evaluator(response="This is the worst response ever!")
print(result)

## Prompt-based evaluators

#### Helpfulness evaluator

In [21]:
from helpfulness import HelpfulnessEvaluator

helpfulness_evaluator = HelpfulnessEvaluator(model_config)

helpfulness_score = helpfulness_evaluator(
    query="What's the meaning of life?", 
    context="Arthur Schopenhauer was the first to explicitly ask the question, in an essay entitled 'Character'.", 
    response="The answer is 42."
)
print(helpfulness_score)

#### JSON accuracy evaluator

In [10]:
import json
from json_schema import JSONSchemaEvaluator

# Load jsons/example.jsonl file here
example_json_schema = json.load(open('jsons/example_schema.json', 'r'))

# Example JSON object
sample_json_data = json.load(open('jsons/poor_output.json', 'r'))

json_schema_evaluator = JSONSchemaEvaluator(model_config)
json_schema_score = json_schema_evaluator(json_output=sample_json_data, schema=example_json_schema)
print(json_schema_score)

{'json_schema': 0.5, 'json_schema_reason': 'The JSON output is mostly compliant with the schema, but it is missing the required "companyName" field in the "companyInfo" object. This omission prevents full compliance with the schema.'}


# Evaluating a dataset

In [11]:
import pandas as pd
from azure.ai.evaluation import evaluate
from pprint import pprint
from model_endpoint import ModelEndpoint
from IPython.display import clear_output

azure_ai_project = {
    "subscription_id": os.environ.get("SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("RG_NAME"),
    "project_name": os.environ.get("PROJECT_NAME"),
}

# Define your evaluators
relevance_evaluator = RelevanceEvaluator(model_config)

# Evaluate the dataset
result = evaluate(
    data="evaluation_dataset.jsonl",
    target=ModelEndpoint(model_config),
    evaluators={
        # Performance and quality evaluators (AI-assisted)
        "relevance": relevance_evaluator,
        "coherence": coherence_evaluator,
        "fluency": fluency_evaluator,
        "groundedness": groundedness_evaluator,
        # Custom evaluators (code and prompt based)
        "helpfulness": helpfulness_evaluator,
    },
    evaluator_config={
        "relevance": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "coherence": {
            "column_mapping": {"response": "${target.response}", "query": "${data.query}"}
        },
        "fluency": {
            "column_mapping": {"response": "${target.response}"}
        },
        "groundedness": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "helpfulness": {
            "column_mapping": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"}
        },
    },
    azure_ai_project=azure_ai_project,
    output_path="./evaluation_results.json",
)
clear_output()

In [12]:
pd.DataFrame(result["rows"])

outputs.query  \
0               What event started on July 28, 1914?   
1      Who was the first person to walk on the moon?   
2  What was the significance of the year 1776 in ...   
3  Which wall fell in 1989, symbolizing the end o...   
4  What ancient city was buried by the eruption o...   
5  Who was the British Prime Minister during Worl...   
6  What was the name of the ship that sank on its...   
7            Which empire was ruled by Genghis Khan?   
8  What was the primary cause of the American Civ...   
9  Which ancient wonder was located in Egypt and ...   

                                    outputs.response  \
0  World War I began on July 28, 1914, when Austr...   
1  The first person to walk on the moon was Neil ...   
2  The year 1776 is highly significant in America...   
3  The Berlin Wall fell in 1989, symbolizing the ...   
4  The ancient city buried by the eruption of Mou...   
5  During World War II, the British Prime Ministe...   
6  The ship that sank on its maiden voyage in 191...   
7  Genghis Khan was the founder and ruler of the ...   
8  The primary cause of the American Civil War wa...   
9  The Great Pyramid of Giza was the ancient wond...   

                                        inputs.query  \
0               What event started on July 28, 1914?   
1      Who was the first person to walk on the moon?   
2  What was the significance of the year 1776 in ...   
3  Which wall fell in 1989, symbolizing the end o...   
4  What ancient city was buried by the eruption o...   
5  Who was the British Prime Minister during Worl...   
6  What was the name of the ship that sank on its...   
7            Which empire was ruled by Genghis Khan?   
8  What was the primary cause of the American Civ...   
9  Which ancient wonder was located in Egypt and ...   

                                      inputs.context  \
0  It involved multiple countries and lasted unti...   
1  The event occurred during the Apollo 11 missio...   
2  A key document was signed declaring independen...   
3       It divided a German city into East and West.   
4  The city's ruins were rediscovered in the 18th...   
5  He is famous for his leadership and speeches, ...   
6  It was deemed 'unsinkable' before it hit an ic...   
7  This empire became the largest contiguous land...   
8  The conflict between the Northern and Southern...   
9  It is the only one of the Seven Wonders of the...   

               inputs.ground_truth  outputs.relevance.relevance  \
0                      World War I                            5   
1                   Neil Armstrong                            5   
2  The Declaration of Independence                            5   
3                  The Berlin Wall                            4   
4                          Pompeii                            5   
5                Winston Churchill                            5   
6                      RMS Titanic                            4   
7                The Mongol Empire                            5   
8                          Slavery                            5   
9        The Great Pyramid of Giza                            5   

   outputs.relevance.gpt_relevance  \
0                                5   
1                                5   
2                                5   
3                                4   
4                                5   
5                                5   
6                                4   
7                                5   
8                                5   
9                                5   

                  outputs.relevance.relevance_reason  \
0  The response is not only accurate and complete...   
1  The response accurately and completely answers...   
2  The response fully addresses the query with ac...   
3  The RESPONSE fully addresses the QUERY with ac...   
4  The response is accurate, complete, and provid...   
5  The response fully addresses the query with ac...   
6  The response is acc